In [43]:
import pandas as pd
import ETL
import importlib
importlib.reload(ETL)
import os
import json

pd.set_option('display.max_columns', None)

path = r"C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Raw"

# Banco de dados org 
df_org = pd.read_csv(path+"/BDorg/jogos.gz", compression="gzip")

# Kaggle Adão
df_KG_AD_jogos = pd.read_csv(path+"/kaggle_AD/jogos.csv", sep=",")
df_KG_AD_gols = pd.read_csv(path+"/kaggle_AD/gols.csv", sep=",")
df_KG_AD_estatisticas = pd.read_csv(path+"/kaggle_AD/estatisticas.csv", sep=",")
df_KG_AD_cartoes = pd.read_csv(path+"/kaggle_AD/cartoes.csv", sep=",")

#GH Hugo Brendow
df_GH_HB_jogos = pd.read_csv(path+"/GH_HugoBrendow/jogos.csv")

#GH Henrique
df_GH_Henr_jogos = pd.read_csv(path+"/GH_Henrique/jogos.csv")
df_GH_Henr_gols = pd.read_csv(path+"/GH_Henrique/gols.csv") 
df_GH_Henr_cartoes = pd.read_csv(path+"/GH_Henrique/cartoes.csv")

#GH Fabrimatt
df_GH_Fab_jogos = pd.read_excel(path+"/GH_Fabrimatt/jogos_stat.xlsx")

#GH Arruda
df_GH_Ard_jogos = ETL.unir_xlsx_em_df(path+"/GH_Arruda/dados/jogos")

##GH Avelar
#df_GH_Ave_jogos = json.load(path+"/GH_Avelar/2003/2003.json")


In [64]:
def ler_json(caminho):
    """
    Lê arquivo JSON e entrega em uma estrutura DataFrame.

    Parâmetros:
    - caminho (str): Caminho completo do arquivo .json.
      Exemplo: r'C:\\Users\\...\\arquivo.json'

    Retorna:
    - df (pd.DataFrame): DataFrame com os dados carregados do JSON.
    """
    with open(caminho, 'r', encoding='utf-8') as arquivo:
        dados = json.load(arquivo)
    df = pd.DataFrame(dados)
    return df

def expandir_json(df: pd.DataFrame, prefixo: str) -> pd.DataFrame:
    """
    Expande a coluna 'pontuacao_geral_{prefixo}' de um DataFrame, que contém dicionários ou JSONs,
    em várias colunas separadas, prefixando os nomes com o valor de 'prefixo'.

    Parâmetros:
        df (pd.DataFrame): DataFrame de entrada contendo a coluna com dados em formato dict/JSON.
        prefixo (str): Prefixo que define o nome da coluna a ser expandida e será usado nas novas colunas.

    Retorna:
        pd.DataFrame: Novo DataFrame com as colunas expandidas e renomeadas com o prefixo.
    """

    coluna_json = df[f'pontuacao_geral_{prefixo}']
    colunas_expandida = pd.json_normalize(coluna_json)
    colunas_expandida.columns = [f'{prefixo}_{col}' for col in colunas_expandida.columns]

    return colunas_expandida

path = r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Raw\GH_Avelar'
arquivos_json = sorted([f for f in os.listdir(path) if f.endswith('.json')])

# aplicando a função para cada arquivo JSON que representa o ano do campeonato
for ano_camp_json in arquivos_json:
    relative_path = path +'\\' + ano_camp_json

    df_json = ler_json(relative_path)
    df_ano_camp = pd.DataFrame()    

    for rodada in range(len(df_json['partidas'])):

        df = pd.DataFrame(df_json['partidas'][rodada])

        pontuacao_mandante = expandir_json(df, 'mandante')
        pontuacao_visitante = expandir_json(df, 'visitante')

        # Juntar tudo em um único DataFrame
        df_rodada = pd.concat([df.drop(['pontuacao_geral_mandante', 'pontuacao_geral_visitante'], axis=1),pontuacao_mandante,pontuacao_visitante], axis=1) 
        df_rodada['rodada'] = df_json['numero'][rodada]

        df_ano_camp = pd.concat([df_ano_camp, df_rodada],axis=0, ignore_index=True)
    df_ano_camp['ano campeonato'] = ano_camp_json.split('.')[0]

    # Mostrar as primeiras linhas
    display(df_ano_camp)



,visitante,resultado,data_partida,placar_visitante,hora_partida,mandante,placar_mandante,estadio,mandante_gols_fora_casa,mandante_empates_fora_casa,mandante_total_jogos,mandante_gols_casa,mandante_jogos_fora_casa,mandante_vitorias_casa,mandante_derrotas_casa,mandante_total_pontos,mandante_empates_casa,mandante_pontos_fora_casa,mandante_total_gols_sofridos,mandante_total_vitorias,mandante_vitorias_fora_casa,mandante_total_derrotas,mandante_pontos_casa,mandante_derrotas_fora_casa,mandante_total_gols_marcados,mandante_jogos_casa,mandante_total_empates,visitante_gols_fora_casa,visitante_empates_fora_casa,visitante_total_jogos,visitante_gols_casa,visitante_jogos_fora_casa,visitante_vitorias_casa,visitante_derrotas_casa,visitante_total_pontos,visitante_empates_casa,visitante_pontos_fora_casa,visitante_total_gols_sofridos,visitante_total_vitorias,visitante_vitorias_fora_casa,visitante_total_derrotas,visitante_pontos_casa,visitante_derrotas_fora_casa,visitante_total_gols_marcados,visitante_jogos_casa,visitante_total_empates,rodada,ano campeonato
0,Goiás,derrota_mandante,21/04/2004,4,16h00,Botafogo,1,Caio Martins,0,0,1,1,0,0,1,0,0,0,4,0,0,1,0,0,1,1,0,4,0,1,0,1,0,0,3,0,3,1,1,1,0,0,0,4,0,0,1,2004
1,Internacional,vitoria_mandante,21/04/2004,0,16h00,Figueirense,1,Orlando Scarpelli,0,0,1,1,0,1,0,3,0,0,0,1,0,0,3,0,1,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,2004
2,Vitória,vitoria_mandante,21/04/2004,0,16h00,São caetano,1,Anacleto Campanella,0,0,1,1,0,1,0,3,0,0,0,1,0,0,3,0,1,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,2004
3,Juventude,vitoria_mandante,21/04/2004,1,18h00,Cruzeiro,2,Ipatingão,0,0,1,2,0,1,0,3,0,0,1,1,0,0,3,0,2,1,0,1,0,1,0,1,0,0,0,0,0,2,0,0,1,0,1,1,0,0,1,2004
4,Atlético mg,empate,21/04/2004,0,18h00,Palmeiras,0,Parque Antártica,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,1,0,1,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,2004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548,Figueirense,vitoria_mandante,19/12/2004,2,16h00,Corinthians,5,Pacaembu,25,7,46,29,23,12,4,74,7,31,54,20,8,12,43,8,54,23,14,26,5,46,31,23,13,3,63,7,17,59,17,4,17,46,14,57,23,12,46,2004
549,Paraná,vitoria_mandante,19/12/2004,1,16h00,Internacional,2,Beira Rio,22,3,46,43,23,15,4,67,4,18,58,20,5,19,49,15,65,23,7,24,7,46,28,23,12,9,54,2,16,73,15,3,22,38,13,52,23,9,46,2004
550,Paysandu,derrota_mandante,19/12/2004,2,16h00,Juventude,1,Alfredo Jaconi,28,3,46,32,23,10,6,70,7,33,66,20,10,16,37,10,60,23,10,14,7,46,42,23,12,4,56,7,13,76,14,2,18,43,14,56,23,14,46,2004
551,Botafogo,empate,19/12/2004,1,16h00,Atlético pr,1,Arena da Baixada,37,6,46,56,23,16,2,86,5,33,56,25,9,10,53,8,93,23,11,24,7,46,38,23,8,4,51,11,16,71,11,3,17,35,13,62,23,18,46,2004


In [62]:
#display(df_ano_camp[df_ano_camp['rodada']==38])
## sao caetano e sao paulo infarto do serginho, jogo parou e retomou, somar as estatisticas e tentar mesclar linha


In [ ]:

for rodada in range(len(df_json['partidas'])):

    df = pd.DataFrame(df_json['partidas'][rodada])

    pontuacao_mandante = ETL.expandir_json(df, 'mandante')
    pontuacao_visitante = ETL.expandir_json(df, 'visitante')

    # Juntar tudo em um único DataFrame
    df_rodada = pd.concat([df.drop(['pontuacao_geral_mandante', 'pontuacao_geral_visitante'], axis=1),pontuacao_mandante,pontuacao_visitante], axis=1) 
    df_rodada['rodada'] = df_json['numero'][rodada]

    df_ano_camp = pd.concat([df_ano_camp, df_rodada],axis=0, ignore_index=True)

# Mostrar as primeiras linhas
display(df_ano_camp)

,visitante,resultado,data_partida,placar_visitante,hora_partida,mandante,placar_mandante,estadio,mandante_gols_fora_casa,mandante_empates_fora_casa,mandante_total_jogos,mandante_gols_casa,mandante_jogos_fora_casa,mandante_vitorias_casa,mandante_derrotas_casa,mandante_total_pontos,mandante_empates_casa,mandante_pontos_fora_casa,mandante_total_gols_sofridos,mandante_total_vitorias,mandante_vitorias_fora_casa,mandante_total_derrotas,mandante_pontos_casa,mandante_derrotas_fora_casa,mandante_total_gols_marcados,mandante_jogos_casa,mandante_total_empates,visitante_gols_fora_casa,visitante_empates_fora_casa,visitante_total_jogos,visitante_gols_casa,visitante_jogos_fora_casa,visitante_vitorias_casa,visitante_derrotas_casa,visitante_total_pontos,visitante_empates_casa,visitante_pontos_fora_casa,visitante_total_gols_sofridos,visitante_total_vitorias,visitante_vitorias_fora_casa,visitante_total_derrotas,visitante_pontos_casa,visitante_derrotas_fora_casa,visitante_total_gols_marcados,visitante_jogos_casa,visitante_total_empates,rodada
0,Vasco,vitoria_mandante,29/03/2003,2,16h00,Guarani,4,Brinco de Ouro,0,0,1,4,0,1,0,3,0,0,2,1,0,0,3,0,4,1,0,2,0,1,0,1,0,0,0,0,0,4,0,0,1,0,1,2,0,0,1
1,Grêmio,vitoria_mandante,29/03/2003,0,16h00,Atlético pr,2,Arena da Baixada,0,0,1,2,0,1,0,3,0,0,0,1,0,0,3,0,2,1,0,0,0,1,0,1,0,0,0,0,0,2,0,0,1,0,1,0,0,0,1
2,Coritiba,empate,30/03/2003,1,16h00,Flamengo,1,Maracanã,0,0,1,1,0,0,0,1,1,0,1,0,0,0,1,0,1,1,1,1,1,1,0,1,0,0,1,0,1,1,0,0,0,0,0,1,0,1,1
3,Paysandu,empate,30/03/2003,2,16h00,Goiás,2,Serra Dourada,0,0,1,2,0,0,0,1,1,0,2,0,0,0,1,0,2,1,1,2,1,1,0,1,0,0,1,0,1,2,0,0,0,0,0,2,0,1,1
4,Ponte preta,empate,30/03/2003,1,16h00,Internacional,1,Beira Rio,0,0,1,1,0,0,0,1,1,0,1,0,0,0,1,0,1,1,1,1,1,1,0,1,0,0,1,0,1,1,0,0,0,0,0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,Cruzeiro,derrota_mandante,14/12/2003,7,16h00,Bahia,0,Fonte Nova,19,4,46,40,23,11,6,46,6,7,92,12,1,24,39,18,59,23,10,41,2,46,61,23,17,1,100,5,44,47,31,14,8,56,7,102,23,7,46
548,Juventude,vitoria_mandante,14/12/2003,0,16h00,Fluminense,1,Maracanã,16,4,46,36,23,11,5,50,7,10,77,13,2,22,40,17,52,23,11,19,4,46,36,23,9,4,50,10,13,70,12,3,20,37,16,55,23,14,46
549,Atlético pr,empate,14/12/2003,2,16h00,Paysandu,2,Mangueirão,21,5,46,53,23,13,3,57,7,11,77,15,2,19,46,16,74,23,12,22,6,46,44,23,14,5,61,4,15,72,17,3,19,46,14,66,23,10,46
550,Santos,empate,14/12/2003,1,18h00,Vasco,1,São Januário,25,8,46,32,23,11,5,54,7,14,69,13,2,18,40,13,57,23,15,38,6,46,55,23,15,2,87,6,36,60,25,10,9,51,7,93,23,12,46
